## Problem 3: How many people live under the dominance area of each shopping center? (additional 5 points available!)

Find out how many people live under the dominance area of each shopping center. 

- Check out week 3 materials for reading in population grid from [HSY wfs](https://www.hsy.fi/fi/asiantuntijalle/avoindata/karttapalvelu/Sivut/Avoimet-rajapinnat.aspx).
- Aggregate your dominance areas into a unified geometries using [`dissolve()`](http://geopandas.org/aggregation_with_dissolve.html#dissolve-example) -function in Geopandas before joining with the population data
- Make the spatial join using `intersect`as the condition

You can freely organize your code into the code cells below.



In [6]:
#Reading Population Data

import geopandas as gpd
from pyproj import CRS
import requests
import geojson

# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format).
# Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
params = dict(service='WFS',
              version='2.0.0',
              request='GetFeature',
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018',
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

In [7]:
pop = pop[['geometry','asukkaita']]
pop = pop.rename(columns={'asukkaita':'population'})
pop.head(n=2)

,geometry,population
0,MULTIPOLYGON Z (((25476499.999 6674248.999 0.0...,108
1,MULTIPOLYGON Z (((25476749.997 6674498.998 0.0...,273


In [9]:
grid = gpd.read_file(r'C:\Users\Maria\Desktop\ISB\GIS Course\exercise-4-master\data\grid\grid.shp')
grid.head(n=2)

,x,y,YKR_ID,pt_r_t_Jum,pt_r_t_Dix,pt_r_t_Myy,pt_r_t_Iti,pt_r_t_For,pt_r_t_Iso,pt_r_t_Ruo,min_t,dominant_s,geometry
0,381875.0,6697880.0,5785640,101.0,102.0,90.0,132.0,110.0,141.0,118.0,90.0,pt_r_t_Myyrmanni,"POLYGON ((382000.000 6697750.000, 381750.000 6..."
1,382125.0,6697880.0,5785641,108.0,109.0,93.0,135.0,113.0,143.0,121.0,93.0,pt_r_t_Myyrmanni,"POLYGON ((382250.000 6697750.000, 382000.000 6..."


In [13]:
#Dissolve Grid by Dominant_s

dissolved = grid.dissolve(by='dominant_s')
dissolved

,geometry,x,y,YKR_ID,pt_r_t_Jum,pt_r_t_Dix,pt_r_t_Myy,pt_r_t_Iti,pt_r_t_For,pt_r_t_Iso,pt_r_t_Ruo,min_t
dominant_s,,,,,,,,,,,,
pt_r_t_Dixi,"MULTIPOLYGON (((387000.000 6677000.000, 386750...",392625.0,6694630.0,5810457,66.0,54.0,85.0,79.0,77.0,106.0,82.0,54.0
pt_r_t_Forum,"MULTIPOLYGON (((385000.000 6669500.000, 384750...",376875.0,6693130.0,5821788,109.0,125.0,106.0,124.0,105.0,131.0,119.0,105.0
pt_r_t_IsoOmena,"MULTIPOLYGON (((373000.000 6665250.000, 372750...",361875.0,6690130.0,5844368,107.0,93.0,73.0,106.0,80.0,72.0,82.0,72.0
pt_r_t_Itis,"MULTIPOLYGON (((391000.000 6667750.000, 390750...",402125.0,6685880.0,5876274,98.0,85.0,97.0,55.0,79.0,100.0,80.0,55.0
pt_r_t_Jumbo,"MULTIPOLYGON (((389500.000 6681500.000, 389250...",384125.0,6694630.0,5810423,104.0,105.0,107.0,142.0,126.0,159.0,134.0,104.0
pt_r_t_Myyrmanni,"MULTIPOLYGON (((384750.000 6676000.000, 384500...",381875.0,6697880.0,5785640,101.0,102.0,90.0,132.0,110.0,141.0,118.0,90.0
pt_r_t_Ruoholahti,"MULTIPOLYGON (((383000.000 6669250.000, 382750...",374375.0,6676380.0,5945731,58.0,55.0,42.0,65.0,38.0,38.0,34.0,34.0


In [20]:
pop.crs = CRS.from_epsg(3879).to_wkt()
dissolved = dissolved.to_crs(pop.crs)

In [26]:
# Spatial join  

join = gpd.sjoin(dissolved, pop)
grouped = join.groupby('dominant_s')
grouped.sum()

,x,y,YKR_ID,pt_r_t_Jum,pt_r_t_Dix,pt_r_t_Myy,pt_r_t_Iti,pt_r_t_For,pt_r_t_Iso,pt_r_t_Ruo,min_t,index_right,population
dominant_s,,,,,,,,,,,,,
pt_r_t_Dixi,295646625.0,5.041056e+09,4375274121,49698.0,40662.0,64005.0,59487.0,57981.0,79818.0,61746.0,40662.0,1778632,201962
pt_r_t_Forum,162056250.0,2.878046e+09,2503368840,46870.0,53750.0,45580.0,53320.0,45150.0,56330.0,51170.0,45150.0,647923,252721
pt_r_t_IsoOmena,244265625.0,4.515838e+09,3944948400,72225.0,62775.0,49275.0,71550.0,54000.0,48600.0,55350.0,48600.0,260791,174969
pt_r_t_Itis,236851625.0,3.937983e+09,3461125386,57722.0,50065.0,57133.0,32395.0,46531.0,58900.0,47120.0,32395.0,1540730,201066
pt_r_t_Jumbo,102177250.0,1.780772e+09,1545572518,27664.0,27930.0,28462.0,37772.0,33516.0,42294.0,35644.0,27664.0,512128,69613
pt_r_t_Myyrmanni,268458125.0,4.708610e+09,4067304920,71003.0,71706.0,63270.0,92796.0,77330.0,99123.0,82954.0,63270.0,732296,203275
pt_r_t_Ruoholahti,76746875.0,1.368658e+09,1218874855,11890.0,11275.0,8610.0,13325.0,7790.0,7790.0,6970.0,6970.0,218319,83754


In [29]:
for index,row in grouped.sum().iterrows():
    print(int(row['population']),' people live in the dominance area of ',index.split(sep='_')[3])

201962  people live in the dominance area of  Dixi
252721  people live in the dominance area of  Forum
174969  people live in the dominance area of  IsoOmena
201066  people live in the dominance area of  Itis
69613  people live in the dominance area of  Jumbo
203275  people live in the dominance area of  Myyrmanni
83754  people live in the dominance area of  Ruoholahti


Remember to print the answers in this notebook :)

### Extra bonus task

Repeat problem 2 and 3 for car accessibility! No extra points available for this, but you can for example start thinking if you want to make the final assignment related to this topic :)